# ===============================================

In [ ]:
#already
!pip install SimpleITK
!pip install monai
!pip install einops
!pip install progress_table
!pip uninstall -y protobuf -q
!pip install protobuf==5.26.1 -q

In [ ]:
import random
import os
import numpy as np
import torch

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"✅ Seed set to {seed}")

# --- THAY SEED ---
CURRENT_SEED = 1
seed_everything(CURRENT_SEED)

In [ ]:
%cd /kaggle/input

# ===================================================================================================
### Testing

In [ ]:
import logging
import os
from datetime import datetime

#Lib for preprocess + train
import SimpleITK as sitk
import torch
from sklearn.model_selection import KFold
from torch.utils.data.dataset import Dataset
import argparse
import random
import pathlib
import time
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import yaml
from monai.data import decollate_batch
from torch.autograd import Variable
import math

import os
import cv2
from glob import glob
import numpy as np
import pandas as pd
import nibabel as nib
from matplotlib import colors
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

#lib for model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch import optim

import wandb

#for training
from functools import partial
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
from monai.transforms import (
    SpatialCrop,
    Activations,
    AsDiscrete,
    Compose,
    EnsureType
)

from monai.losses import DiceLoss #Minh co can code lai ham loss khong?
import monai.transforms as transforms
from monai.transforms import NormalizeIntensity

In [ ]:
#đáng lẽ hai cái này là hai cái khác nhau nhưng vì ở đây mình dùng bộ data đã có sẵn traningdata nên trùng

BRATS_TRAIN_FOLDERS = "/kaggle/input/brats2021/BraTS_2021_TrainingData/BraTS_2021_TrainingData"
DATA_PATH = '/kaggle/input/brats2021/BraTS_2021_TrainingData'
VALI_PATH = '/kaggle/input/brats2021/BraTS_2021_ValidationData/BraTS_2021_ValidationData'

DATA_TYPES = ['t1', 't1ce', 't2', 'flair', 'seg']
MASK_LABELS = ['Not Tumor', 'Non-Enhancing Tumor Core', 'Peritumoral Edema', 'GD-Enhancing Tumor']
MASK_VALUES = [0, 1, 2, 4]
#data_paths = {dt: sorted(glob(DATA_PATH + '/*/*.' + dt + '.nii')) for dt in DATA_TYPES}
# data_paths = {dt: sorted(glob(DATA_PATH + '/**/*_{}.nii'.format(dt))) for dt in DATA_TYPES}
# data_paths['seg'] = sorted(data_paths['seg'])
# data_paths_train = {dt: sorted(glob(os.path.join(DATA_PATH, '**', f'*.{dt}.nii'), recursive=True)) for dt in DATA_TYPES}
# data_paths_vali = {dt: sorted(glob(os.path.join(VALI_PATH, '**', f'*.{dt}.nii'), recursive=True)) for dt in DATA_TYPES}

# # Merge train and validation paths for each data type
# data_paths = {dt: data_paths_train[dt] + data_paths_vali[dt] for dt in DATA_TYPES}

# data_paths['seg'] = sorted(data_paths['seg'])
# for key, vals in data_paths.items():
#     print('[TRAIN] Number of {} images: {}'.format(key,  len(vals)))

In [ ]:
#preprocessing function
"""functions to correctly pad or crop non uniform sized MRI (before batching in the dataloader).
"""
def get_brats_folder(on="train"):
    if on == "train":
        return BRATS_TRAIN_FOLDERS
    else:
        return VALI_PATH
def pad_or_crop_image(image, seg=None, target_size=(128, 144, 144)):
    c, z, y, x = image.shape
    z_slice, y_slice, x_slice = [get_crop_slice(target, dim) for target, dim in zip(target_size, (z, y, x))]
    image = image[:, z_slice, y_slice, x_slice]
    if seg is not None:
        seg = seg[:, z_slice, y_slice, x_slice]
    todos = [get_left_right_idx_should_pad(size, dim) for size, dim in zip(target_size, [z, y, x])]
    padlist = [(0, 0)]  # channel dim
    for to_pad in todos:
        if to_pad[0]:
            padlist.append((to_pad[1], to_pad[2]))
        else:
            padlist.append((0, 0))
    image = np.pad(image, padlist)
    if seg is not None:
        seg = np.pad(seg, padlist)
        return image, seg
    return image

def get_left_right_idx_should_pad(target_size, dim):
    if dim >= target_size:
        return [False]
    elif dim < target_size:
        pad_extent = target_size - dim
        left = random.randint(0, pad_extent)
        right = pad_extent - left
        return True, left, right

def get_crop_slice(target_size, dim):
    if dim > target_size:
        crop_extent = dim - target_size
        left = random.randint(0, crop_extent)
        right = crop_extent - left
        return slice(left, dim - right)
    elif dim <= target_size:
        return slice(0, dim)
def get_crop_slice2(target_size, dim):
    if dim > target_size:
        start = (dim - target_size) // 2
        end = start + target_size
        return slice(start, end)
    elif dim <= target_size:
        return slice(0, dim)
def pad_or_crop_image_label(image, seg=None, target_size=(128, 128, 128)):
    c, z, y, x = image.shape
    z_slice, y_slice, x_slice = [get_crop_slice2(target, dim) for target, dim in zip(target_size, (z, y, x))]
    image = image[:, z_slice, y_slice, x_slice]
    if seg is not None:
        seg = seg[:, z_slice, y_slice, x_slice]
    
    # Pad nếu cần
    # todos = [get_left_right_idx_should_pad(size, dim) for size, dim in zip(target_size, [z, y, x])]
    # padlist = [(0, 0)]  # channel dim
    # for to_pad in todos:
    #     if to_pad[0]:
    #         padlist.append((to_pad[1], to_pad[2]))
    #     else:
    #         padlist.append((0, 0))
    
    # # Pad ảnh để đạt được target_size
    # image = np.pad(image, padlist)
    if seg is not None:
        # seg = np.pad(seg, padlist)
        return image, seg
    return image


def normalize(image):
    """Basic min max scaler.
    """
    min_ = np.min(image)
    max_ = np.max(image)
    scale = max_ - min_
    image = (image - min_) / scale
    return image

def irm_min_max_preprocess(image, low_perc=1, high_perc=99):
    """Main pre-processing function used for the challenge (seems to work the best).

    Remove outliers voxels first, then min-max scale.

    Warnings
    --------
    This will not do it channel wise!!
    """

    non_zeros = image > 0
    low, high = np.percentile(image[non_zeros], [low_perc, high_perc])
    image = np.clip(image, low, high)
    image = normalize(image)
    return image

def zscore_normalise(img):
    # Đảm bảo rằng img có kiểu dữ liệu float
    img = img.astype(np.float32)  # Nếu img là ndarray
    # img = img.to(torch.float32)  # Nếu img là tensor
    
    slices = slice(None)
    img[slices] = (img[slices] - np.mean(img[slices])) / np.std(img[slices])
    return img

In [ ]:
class Brats(Dataset):
    def __init__(self, patients_dir, benchmarking=False, training=True, data_aug=False,
                 no_seg=False, normalisation="zscore", normal=False):
        super(Brats, self).__init__()
        self.benchmarking = benchmarking
        self.normalisation = normalisation
        self.data_aug = data_aug
        self.training = training
        self.datas = []
        self.validation = no_seg
        self.normal = normal
        self.patterns = [".t1", ".t1ce", ".t2", ".flair"]
        self.target_spacing = [1.0, 1.0, 1.0]
        self.roi_size = [128, 128, 128]
        if self.training:
            self.transform = transforms.Compose([
                transforms.RandFlipd(keys=["image", "label"], prob=0.1, spatial_axis=0), #aug
                transforms.RandFlipd(keys=["image", "label"], prob=0.1, spatial_axis=1), #aug
                transforms.RandFlipd(keys=["image", "label"], prob=0.1, spatial_axis=2), #aug
                # transforms.NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
                transforms.RandScaleIntensityd(keys="image", factors=0.1, prob=1.0), #aug
                transforms.RandShiftIntensityd(keys="image", offsets=0.1, prob=1.0), #aug
            ])
        else:
            # Validation/Inference pipeline
            self.transform = transforms.Compose([
                transforms.CropForegroundd(
                    keys=["image", "label"],
                    source_key="image",
                    k_divisible=self.roi_size,
                )
            ])
        
        if not no_seg:
            self.patterns += [".seg"]
        for patient_dir in patients_dir:
            patient_id = patient_dir.name
            # Construct paths for each modality
            paths = [patient_dir / f"{patient_id}{value}.nii" for value in self.patterns]
            
            # Add patient data
            patient = dict(
                id=patient_id,
                t1=paths[0], t1ce=paths[1], t2=paths[2], flair=paths[3],
                seg=paths[4] if not no_seg else None
            )
            self.datas.append(patient)


    
        
    def __getitem__(self, idx):
        _patient = self.datas[idx]
        
        patient_image = {key: self.load_nii(_patient[key]) for key in _patient if key not in ["id", "seg"]} 
        #-> load t1 t1w t2 flair
        if _patient["seg"] is not None:
            patient_label = self.load_nii(_patient["seg"])
            et = patient_label == 4 # ET (Enhancing Tumor) - label 4
            et_present = 1 if np.sum(et) >= 1 else 0
            tc = np.logical_or(patient_label == 4, patient_label == 1) #TC (Tumor Core) - label 1, 4
            wt = np.logical_or(tc, patient_label == 2) # WT (Whole Tumor) label 1, 2, 4
            patient_label = np.stack([et, tc, wt])
        else:
            patient_label = np.zeros(patient_image.shape)  # placeholders, not gonna use it
            et_present = 0

        if self.normal == False: 
            if self.normalisation == "minmax":
                patient_image = {key: irm_min_max_preprocess(patient_image[key]) for key in patient_image}
            elif self.normalisation == "zscore":
                patient_image = {key: zscore_normalise(patient_image[key]) for key in patient_image}
        else:
            # patient_image = {key: zscore_normalise(patient_image[key]) for key in patient_image}
            normalize_intensity = NormalizeIntensity(nonzero=True, channel_wise=True)
            patient_image = {key: normalize_intensity(patient_image[key]) for key in patient_image}  # Chuẩn hóa cường độ cho mỗi ảnh

        patient_image = np.stack([patient_image[key] for key in patient_image])

        if self.training:
            z_indexes, y_indexes, x_indexes = np.nonzero(np.sum(patient_image, axis=0) != 0)
            zmin, ymin, xmin = [max(0, int(np.min(arr) - 1)) for arr in (z_indexes, y_indexes, x_indexes)]
            zmax, ymax, xmax = [int(np.max(arr) + 1) for arr in (z_indexes, y_indexes, x_indexes)]
            patient_image = patient_image[:, zmin:zmax, ymin:ymax, xmin:xmax]
            patient_label = patient_label[:, zmin:zmax, ymin:ymax, xmin:xmax]              
            patient_image, patient_label = pad_or_crop_image(patient_image, patient_label, target_size=(128, 128, 128))

            if self.normal == True:
                data_dict = {
                    "image": patient_image,
                    "label": patient_label,
                }
                transformed = self.transform(data_dict)
    
                patient_image = transformed["image"]
                patient_label = transformed["label"]
        else:
            z_indexes, y_indexes, x_indexes = np.nonzero(np.sum(patient_image, axis=0) != 0)
            zmin, ymin, xmin = [max(0, int(np.min(arr) - 1)) for arr in (z_indexes, y_indexes, x_indexes)]
            zmax, ymax, xmax = [int(np.max(arr) + 1) for arr in (z_indexes, y_indexes, x_indexes)]
            patient_image = patient_image[:, zmin:zmax, ymin:ymax, xmin:xmax]
            patient_label = patient_label[:, zmin:zmax, ymin:ymax, xmin:xmax]  
            patient_image, patient_label = pad_or_crop_image_label(patient_image, patient_label, target_size=(128, 128, 128))


        # Tiến hành các bước tiếp theo với transformed
        patient_image, patient_label = patient_image.astype("float16"), patient_label.astype("bool")
        patient_image, patient_label = [torch.from_numpy(x) for x in [patient_image, patient_label]]

        return dict(patient_id=_patient["id"],
                    image=patient_image, label=patient_label,
                    seg_path=str(_patient["seg"]) if self.training else str(_patient["t1"]),
                    crop_indexes=((zmin, zmax), (ymin, ymax), (xmin, xmax)),
                    et_present=et_present,
                    supervised=True,
                    idx=idx,
                    )

    @staticmethod
    def load_nii(path_folder):
        return sitk.GetArrayFromImage(sitk.ReadImage(str(path_folder)))

    def __len__(self):
        return len(self.datas)

In [ ]:
##### 
"""
Should check this

Get full data
"""


def get_datasets(seed, on="train", fold_number=0, normalisation="zscore", use_fold = False):
    base_folder_train = pathlib.Path(get_brats_folder(on)).resolve()
    base_folder_vali = pathlib.Path(get_brats_folder("a")).resolve()
    assert base_folder_train.exists()
    patients_dir_train = sorted([x for x in base_folder_train.iterdir() if x.is_dir()])
    patients_dir_vali = sorted([x for x in base_folder_vali.iterdir() if x.is_dir()])


    if use_fold == True:
        kfold = KFold(5, shuffle=True, random_state=seed)
    
        splits = list(kfold.split(patients_dir_train))
        train_idx, test_idx = splits[fold_number]
    
    
        train = [patients_dir_train[i] for i in train_idx]
        test = [patients_dir_train[i] for i in test_idx]
        train_dataset = Brats(train, training=True, normalisation=normalisation, normal = True)
        test_dataset = Brats(test, training=False, benchmarking=True, normalisation=normalisation, normal = True)
    else:
        train_dataset = Brats(patients_dir_train, training=True, normalisation=normalisation, normal = True)
        test_dataset = Brats(patients_dir_vali, training=False, benchmarking=True, normalisation=normalisation, normal = True)

    return train_dataset, test_dataset

full_train_dataset, val_dataset = get_datasets(123, fold_number=0)
print(len(full_train_dataset), len(val_dataset))

In [ ]:
"""
Create loader for neural network
"""


train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=2, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=2)

print("Train dataset number of batch:", len(train_loader))
print("Val dataset number of batch:", len(val_loader))

In [ ]:
"""
Xem lại định dạng dữ liệu đầu vào của mô hình

"""

for batch in train_loader:
    # Assuming your input data is a 4D tensor (batch_size, channels, height, width)
    
    data_shape = batch['image'].shape
    label_shape = batch['label'].shape
    print("Data shape in the first batch:", data_shape)
    print("Label shape in the first batch:", label_shape)
    break  # Print only the first batch

In [ ]:
# import matplotlib.pyplot as plt
# label_colors = ['red','yellow', 'green']
# color_values = {'red': (255, 0, 0, 255),  'yellow': (255, 255, 0, 255),'green': (0, 255, 0, 255)}

# background_color = (0, 0, 0, 255)
# i=0
# # Get a single batch from the DataLoader
# for batch in train_loader:

#     print(batch['patient_id'])
#     image_sample = batch['image']
#     label_sample = batch['label']
#     image_sample_np = image_sample.numpy()
#     label_sample_np = label_sample.numpy()


#     # Select the central slice along the z-axis (depth) for visualization
#     z_slice =  image_sample_np.shape[2] // 2 +2 # between

#     # Plot each channel of the image sample
#     num_channels = image_sample_np.shape[1]
#     fig, axes = plt.subplots(1, num_channels, figsize=(15, 5))

#     for channel in range(num_channels):
#         axes[channel].imshow(image_sample_np[0, channel, z_slice], cmap='gray')
#         axes[channel].set_title(f"Image Channel {channel + 1}")


#     plt.tight_layout()
#     plt.show()

#     # Plot each channel of the label sample (assuming 3 channels for one-hot encoded segmentation)
#     num_channels_labels = label_sample_np.shape[1]
#     fig, axes = plt.subplots(1, num_channels_labels, figsize=(15, 5))

#     for channel in range(num_channels_labels):
#         axes[channel].imshow(label_sample_np[0, channel, z_slice], cmap='gray')
#         axes[channel].set_title(f"Label Channel {channel + 1}")


#     plt.tight_layout()
#     plt.show()



#     image_sample_np = np.full((label_sample_np.shape[3], label_sample_np.shape[4], 4), background_color, dtype=np.uint8)
#     # Combine the label channels with different colors
#     num_channels_labels = label_sample_np.shape[1]
#     for channel in range(num_channels_labels-1, -1, -1):
#         label_channel = label_sample_np[0, channel, z_slice]

#         # Overlay the label with a unique color
#         label_color = label_colors[channel % len(label_colors)]
#         color_value = color_values[label_color]

#         # Create a mask for the current label channel
#         label_mask = label_channel > 0

#         # Apply the color with alpha channel to the corresponding pixels in the composite label
#         image_sample_np[label_mask] = color_value

#     # Plot the composite label image
#     plt.figure(figsize=(10, 5))
#     plt.imshow(image_sample_np)
#     plt.title("Composite Label")
#     plt.show()
#     i+=1
#     if i == 10:
#         break

# ===================================================================================================


## Set up model

In [ ]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import math 

from torch.autograd import Variable
from einops import rearrange, repeat 
from einops.layers.torch import Rearrange
import einops
import numpy as np

# from timm.models.layers import DropPath

# Wavelet Blocks

In [ ]:
import torch
import torch.nn as nn
import numpy as np

def get_wav_3d(in_channels, pool=True):
    # 3D Haar wavelet filters
    s = 1 / np.sqrt(2)
    harr_wav_L = s * np.ones((1, 2))
    harr_wav_H = s * np.ones((1, 2))
    harr_wav_H[0, 0] = -harr_wav_H[0, 0]

    # 3D filter banks (2x2x2 kernels)
    LLL = np.kron(np.kron(harr_wav_L, harr_wav_L), harr_wav_L).reshape(1, 1, 2, 2, 2)
    LLH = np.kron(np.kron(harr_wav_L, harr_wav_L), harr_wav_H).reshape(1, 1, 2, 2, 2)
    LHL = np.kron(np.kron(harr_wav_L, harr_wav_H), harr_wav_L).reshape(1, 1, 2, 2, 2)
    LHH = np.kron(np.kron(harr_wav_L, harr_wav_H), harr_wav_H).reshape(1, 1, 2, 2, 2)
    HLL = np.kron(np.kron(harr_wav_H, harr_wav_L), harr_wav_L).reshape(1, 1, 2, 2, 2)
    HLH = np.kron(np.kron(harr_wav_H, harr_wav_L), harr_wav_H).reshape(1, 1, 2, 2, 2)
    HHL = np.kron(np.kron(harr_wav_H, harr_wav_H), harr_wav_L).reshape(1, 1, 2, 2, 2)
    HHH = np.kron(np.kron(harr_wav_H, harr_wav_H), harr_wav_H).reshape(1, 1, 2, 2, 2)

    filters = [LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH]
    filter_tensors = [torch.from_numpy(f).float() for f in filters]  # Shape: [1, 1, 2, 2, 2]

    if pool:
        net = nn.Conv3d
    else:
        net = nn.ConvTranspose3d

    subbands = []
    for f in filter_tensors:
        conv = net(in_channels, in_channels, kernel_size=2, stride=2, padding=0, bias=False, groups=in_channels)
        conv.weight.requires_grad = False
        # More efficient expansion - using repeat_interleave for grouped convolution
        f_expanded = f.repeat_interleave(in_channels, dim=0).view(in_channels, 1, 2, 2, 2)
        conv.weight.data = f_expanded
        subbands.append(conv)
    
    return subbands

class WavePool3D(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.LLL, self.LLH, self.LHL, self.LHH, self.HLL, self.HLH, self.HHL, self.HHH = get_wav_3d(in_channels, pool=True)

    def forward(self, x):
        return [self.LLL(x), self.LLH(x), self.LHL(x), self.LHH(x), self.HLL(x), self.HLH(x), self.HHL(x), self.HHH(x)]

class WaveUnpool3D(nn.Module):
    def __init__(self, in_channels, option_unpool='sum'):
        super().__init__()
        self.in_channels = in_channels
        self.option_unpool = option_unpool
        self.LLL, self.LLH, self.LHL, self.LHH, self.HLL, self.HLH, self.HHL, self.HHH = get_wav_3d(in_channels, pool=False)

    def forward(self, subbands, include_low=True):
        # Unpack the list of subbands
        LLL, LLH, LHL, LHH, HLL, HLH, HHL, HHH = subbands
        if self.option_unpool == 'sum':
            high_freq_sum = (self.LLH(LLH) + self.LHL(LHL) + self.LHH(LHH) + 
                             self.HLL(HLL) + self.HLH(HLH) + self.HHL(HHL) + 
                             self.HHH(HHH))
            if include_low:
                return self.LLL(LLL) + high_freq_sum
            else:
                return high_freq_sum
                
        elif self.option_unpool == 'cat':
            if include_low:
                return torch.cat([self.LLL(LLL), self.LLH(LLH), self.LHL(LHL), self.LHH(LHH),
                              self.HLL(HLL), self.HLH(HLH), self.HHL(HHL), self.HHH(HHH)], dim=1)
            else:
                return torch.cat([self.LLH(LLH), self.LHL(LHL), self.LHH(LHH),
                              self.HLL(HLL), self.HLH(HLH), self.HHL(HHL), self.HHH(HHH)], dim=1)
        else:
            raise NotImplementedError


class WaveletAttentionFusion(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # Project high-frequency features to out_channels
        self.high_freq_proj = nn.Conv3d(in_channels * 7, out_channels, kernel_size=1, bias=False)
        # Attention mechanism for high-frequency features
        self.attn = nn.Sequential(
            nn.Conv3d(out_channels, out_channels, kernel_size=1, bias=False),
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv3d(out_channels, 1, kernel_size=1),  # Spatial attention
            nn.Sigmoid()
        )
        # Fuse main and attended high-frequency features
        self.fuse = nn.Sequential(
            nn.Conv3d(out_channels * 2, out_channels, kernel_size=1, bias=False),
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x, high_freq):
        # high_freq: list of 7 tensors [B, in_channels, D, H, W]
        high_freq_cat = torch.cat(high_freq, dim=1)  # [B, in_channels*7, D, H, W]
        high_freq_feat = self.high_freq_proj(high_freq_cat)  # [B, out_channels, D, H, W]
        attn_map = self.attn(high_freq_feat)  # [B, 1, D, H, W]
        high_freq_attended = high_freq_feat * attn_map  # [B, out_channels, D, H, W]
        fused = torch.cat([x, high_freq_attended], dim=1)  # [B, out_channels*2, D, H, W]
        out = self.fuse(fused)
        return out

## Base

In [ ]:
class LayerNormProxy3d(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim) #LayerNorm chuẩn hóa chiều cuối cùng

    def forward(self, x):
        x = einops.rearrange(x, 'b c d h w -> b d h w c')
        x = self.norm(x)
        return einops.rearrange(x, 'b d h w c -> b c d h w')


In [ ]:
class DoubleConv(nn.Module):
    """(Conv3D -> BN -> ReLU) * 2"""
    def __init__(self, in_channels, out_channels, num_groups=16, kernel_size=3, padding=1, stride=1, bias=True):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, bias=bias),
            nn.GroupNorm(num_groups = num_groups, num_channels = out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, bias=bias),
            nn.GroupNorm(num_groups = num_groups, num_channels = out_channels),
            nn.LeakyReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)

In [ ]:
class ProjectExciteLayer(nn.Module):
    """
    Project & Excite Module, with wave-modulated and adaptively weighted
    combination of spatially-squeezed features.
    """

    def __init__(self, num_channels, reduction_ratio=2, mlp_hidden_ratio=1):
        """
        :param num_channels: No of input channels
        :param reduction_ratio: By how much num_channels should be reduced for the excitation bottleneck
        :param mlp_hidden_ratio: Ratio for hidden dim in MLPs generating theta and mixing weights
        """
        super(ProjectExciteLayer, self).__init__()
        self.num_channels = num_channels
        num_channels_reduced = num_channels // reduction_ratio
        mlp_hidden_dim = num_channels * mlp_hidden_ratio

        # --- MLPs for Theta and Mixing Weights ---
        # These will operate on a global context vector derived from the input

        # MLP to generate theta parameters for wave modulation (shared for w, h, d)
        # Outputs C parameters, which will be broadcasted
        self.theta_mlp = nn.Sequential(
            nn.Linear(num_channels, mlp_hidden_dim),
            nn.LayerNorm(mlp_hidden_dim), # Common practice in "wave" examples
            nn.GELU(),
            nn.Linear(mlp_hidden_dim, num_channels)
        )

        # MLP to generate mixing weights for the three directions (w, h, d)
        # Outputs 3*C parameters for per-channel weighting of each direction
        self.reweight_mlp = nn.Sequential(
            nn.Linear(num_channels, mlp_hidden_dim),
            nn.LayerNorm(mlp_hidden_dim), # Common practice in "wave" examples
            nn.GELU(),
            nn.Linear(mlp_hidden_dim, num_channels * 3) # 3 weights per channel
        )

        # --- Original P&E Components ---
        self.relu = nn.ReLU()
        self.conv_c = nn.Conv3d(in_channels=num_channels, out_channels=num_channels_reduced, kernel_size=1, stride=1)
        self.conv_cT = nn.Conv3d(in_channels=num_channels_reduced, out_channels=num_channels, kernel_size=1, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        """
        :param input_tensor: X, shape = (batch_size, num_channels, D, H, W)
        :return: output tensor
        """
        batch_size, num_channels, D, H, W = input_tensor.size()

        # --- 1. Original Squeeze Operations ---
        squeeze_tensor_w = F.adaptive_avg_pool3d(input_tensor, (1, 1, W)) # (B, C, 1, 1, W)
        squeeze_tensor_h = F.adaptive_avg_pool3d(input_tensor, (1, H, 1)) # (B, C, 1, H, 1)
        squeeze_tensor_d = F.adaptive_avg_pool3d(input_tensor, (D, 1, 1)) # (B, C, D, 1, 1)

        # --- 2. Generate Global Context for MLPs ---
        global_context = F.adaptive_avg_pool3d(input_tensor, (1, 1, 1)).view(batch_size, num_channels) # (B, C)

        # --- 3. Generate Theta for Wave Modulation ---
        # Theta has shape (B, C), will be reshaped for broadcasting with squeezed tensors
        theta = self.theta_mlp(global_context) # (B, C)
        # Reshape theta for broadcasting with (B, C, D, H, W) style features
        theta_bc111 = theta.view(batch_size, num_channels, 1, 1, 1)

        # --- 4. Apply Wave Modulation to Squeezed Tensors ---
        # s_mod = s * cos(theta) + s * sin(theta)
        # theta_bc111 will broadcast correctly with squeeze_tensor_w, _h, _d
        s_w_mod = squeeze_tensor_w * torch.cos(theta_bc111) + squeeze_tensor_w * torch.sin(theta_bc111)
        s_h_mod = squeeze_tensor_h * torch.cos(theta_bc111) + squeeze_tensor_h * torch.sin(theta_bc111)
        s_d_mod = squeeze_tensor_d * torch.cos(theta_bc111) + squeeze_tensor_d * torch.sin(theta_bc111)

        # --- 5. Generate Adaptive Mixing Weights ---
        # reweight_mlp outputs (B, 3*C) for per-channel weights for 3 directions
        mixing_weights_flat = self.reweight_mlp(global_context) # (B, 3*C)
        # Reshape to (B, C, 3) and apply softmax over the 3 directions for each channel
        mixing_weights_per_channel = mixing_weights_flat.view(batch_size, num_channels, 3).softmax(dim=2) # (B, C, 3)

        alpha_w = mixing_weights_per_channel[:, :, 0].view(batch_size, num_channels, 1, 1, 1) # (B, C, 1, 1, 1)
        alpha_h = mixing_weights_per_channel[:, :, 1].view(batch_size, num_channels, 1, 1, 1) # (B, C, 1, 1, 1)
        alpha_d = mixing_weights_per_channel[:, :, 2].view(batch_size, num_channels, 1, 1, 1) # (B, C, 1, 1, 1)

        # --- 6. Wave-Modulated and Weighted Combination ---
        # Each alpha (B,C,1,1,1) will multiply its corresponding modulated squeezed tensor (B,C,D',H',W')
        final_squeeze_tensor = (alpha_w * s_w_mod.view(batch_size, num_channels, 1, 1, W) +
                                alpha_h * s_h_mod.view(batch_size, num_channels, 1, H, 1) +
                                alpha_d * s_d_mod.view(batch_size, num_channels, D, 1, 1))
        # The .view() calls ensure the components are structured as in the original P&E's sum,
        # which is important for how self.conv_c processes them.

        # --- 7. Original Excitation Path ---
        excitation = self.sigmoid(self.conv_cT(self.relu(self.conv_c(final_squeeze_tensor))))

        # --- 8. Apply Excitation ---
        output_tensor = torch.mul(input_tensor, excitation)

        return output_tensor

In [ ]:
class LKA(nn.Module):
    def __init__(self, dim, k_size):
        super().__init__()
        self.k_size = k_size
        
        self.conv_w = nn.Conv3d(dim, dim, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2), groups=dim)   
        self.gn = nn.GroupNorm(num_groups=dim, num_channels=dim)
        self.irelu = nn.ReLU(inplace=True)
        self.conv_spatial = nn.Conv3d(dim, dim, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(9, 9, 9), groups=dim, dilation=(3, 3, 3))
        self.conv = nn.Conv3d(dim, dim, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        u = x.clone()

        attn = self.conv_w(x)

        attn = self.gn(attn)

        attn = self.irelu(attn)

        attn = self.conv_spatial(attn)

        attn = self.conv(attn)

        attn = self.sigmoid(attn)

        attn = u*attn
        u = u + attn
        return u

In [ ]:
class Down2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.encode1 = nn.Sequential(
            nn.MaxPool3d(kernel_size=2, stride=2),
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False),
            nn.InstanceNorm3d(num_features=out_channels),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
        )
        self.spatial_gating_unit = LKA(dim = out_channels, k_size=3)
        self.conv = nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.INorm = nn.InstanceNorm3d(num_features=in_channels)
        self.LR = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.PE = ProjectExciteLayer(num_channels = out_channels)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.encode1(x)
        x = self.spatial_gating_unit(x)
        x1 = self.INorm(x)
        x1 = self.PE(x1)
        x1 = self.sigmoid(x1)
        x = x1 * x # element-wise multiplied
        return x


class Down2W(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.wave_pool = WavePool3D(in_channels)
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False),
            nn.InstanceNorm3d(num_features=out_channels),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
        )
        self.wavelet_fusion = WaveletAttentionFusion(in_channels, out_channels)
        self.spatial_gating_unit = LKA(dim=out_channels, k_size=3)
        self.INorm = nn.InstanceNorm3d(num_features=out_channels)
        self.PE = ProjectExciteLayer(num_channels=out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        subbands = self.wave_pool(x)
        LLL = subbands[0]
        high_freq = subbands[1:]
        x_main = self.conv(LLL)
        x = self.wavelet_fusion(x_main, high_freq)
        x = self.spatial_gating_unit(x)
        x1 = self.INorm(x)
        x1 = self.PE(x1)
        x1 = self.sigmoid(x1)
        x = x1 * x
        return x, subbands

In [ ]:
class Up1W(nn.Module):
    def __init__(self, in_channels, out_channels, trilinear=False):
        super().__init__()
        self.upT = nn.ConvTranspose3d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)
        self.wave_unpool = WaveUnpool3D(out_channels, option_unpool='cat')
        
        self.wavelet_proj = nn.Sequential(
            nn.Conv3d(out_channels * 7, out_channels, kernel_size=1, 
                      groups=out_channels, bias=False),
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(inplace=True)
        )
        
        # Simplified three-way fusion attention
        self.fusion_attention = nn.Sequential(
            nn.Conv3d(out_channels * 3, out_channels, kernel_size=1),
            nn.InstanceNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, 3, kernel_size=1),
            nn.Sigmoid()
        )
        
        self.conv_out = nn.Sequential(
            nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1,bias=False), 
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            
            nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm3d(out_channels),
            nn.LeakyReLU(negative_slope=0.01, inplace=True)
        )
        
        self.PE = ProjectExciteLayer(num_channels=out_channels)

    def forward(self, x1, x2, subbands):
        # Upsample and reconstruct
        up1 = self.upT(x1)
        wave_up = self.wave_unpool(subbands, include_low=False)
        wave_up = self.wavelet_proj(wave_up)
        
        # Three-way attention fusion
        fusion_input = torch.cat([up1, x2, wave_up], dim=1)
        attention_weights = self.fusion_attention(fusion_input)  # [B, 3, D, H, W]
        
        # Weighted fusion
        fused = (attention_weights[:, 0:1] * up1 + 
                attention_weights[:, 1:2] * x2 + 
                attention_weights[:, 2:3] * wave_up)
        
        # Apply PE enhancement and final processing
        pe_weights = torch.sigmoid(self.PE(fused))
        enhanced = fused * pe_weights
        
        # Final convolutions with residual
        output = self.conv_out(enhanced)
        return output + enhanced

## Wave Block

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import DropPath, trunc_normal_
import math

class Mlp3D(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.act = act_layer()
        self.drop = nn.Dropout(drop)
        self.fc1 = nn.Conv3d(in_features, hidden_features, 1, 1)
        self.fc2 = nn.Conv3d(hidden_features, out_features, 1, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class PATM3D(nn.Module):
    def __init__(self, in_dim, out_dim, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., mode='fc'):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.fc_h = nn.Conv3d(in_dim, out_dim, 1, 1, bias=qkv_bias)
        self.fc_w = nn.Conv3d(in_dim, out_dim, 1, 1, bias=qkv_bias)
        self.fc_d = nn.Conv3d(in_dim, out_dim, 1, 1, bias=qkv_bias)
        self.fc_c = nn.Conv3d(in_dim, out_dim, 1, 1, bias=qkv_bias)
        self.tfc_h = nn.Conv3d(2 * out_dim, out_dim, (1, 1, 7), stride=1, padding=(0, 0, 7//2), groups=out_dim, bias=False)
        self.tfc_w = nn.Conv3d(2 * out_dim, out_dim, (1, 7, 1), stride=1, padding=(0, 7//2, 0), groups=out_dim, bias=False)
        self.tfc_d = nn.Conv3d(2 * out_dim, out_dim, (7, 1, 1), stride=1, padding=(7//2, 0, 0), groups=out_dim, bias=False)
        self.reweight = Mlp3D(out_dim, out_dim // 4, out_dim * 4)
        self.proj = nn.Conv3d(out_dim, out_dim, 1, 1, bias=True)
        self.proj_drop = nn.Dropout(proj_drop)
        self.mode = mode
        if mode == 'fc':
            self.theta_h_conv = nn.Sequential(
                nn.Conv3d(in_dim, out_dim, 1, 1, bias=True),
                nn.GroupNorm(num_groups=out_dim, num_channels=out_dim),
                nn.ReLU()
            )
            self.theta_w_conv = nn.Sequential(
                nn.Conv3d(in_dim, out_dim, 1, 1, bias=True),
                nn.GroupNorm(num_groups=out_dim, num_channels=out_dim),
                nn.ReLU()
            )
            self.theta_d_conv = nn.Sequential(
                nn.Conv3d(in_dim, out_dim, 1, 1, bias=True),
                nn.GroupNorm(num_groups=out_dim, num_channels=out_dim),
                nn.ReLU()
            )
        else:
            self.theta_h_conv = nn.Sequential(
                nn.Conv3d(in_dim, out_dim, 3, stride=1, padding=1, groups=in_dim, bias=False),
                nn.GroupNorm(num_groups=out_dim, num_channels=out_dim),
                nn.ReLU()
            )
            self.theta_w_conv = nn.Sequential(
                nn.Conv3d(in_dim, out_dim, 3, stride=1, padding=1, groups=in_dim, bias=False),
                nn.GroupNorm(num_groups=out_dim, num_channels=out_dim),
                nn.ReLU()
            )
            self.theta_d_conv = nn.Sequential(
                nn.Conv3d(in_dim, out_dim, 3, stride=1, padding=1, groups=in_dim, bias=False),
                nn.GroupNorm(num_groups=out_dim, num_channels=out_dim),
                nn.ReLU()
            )

    def forward(self, x):
        B, C, D, H, W = x.shape
        theta_h = self.theta_h_conv(x)
        theta_w = self.theta_w_conv(x)
        theta_d = self.theta_d_conv(x)
        x_h = self.fc_h(x)
        x_w = self.fc_w(x)
        x_d = self.fc_d(x)
        x_h = torch.cat([x_h * torch.cos(theta_h), x_h * torch.sin(theta_h)], dim=1)
        x_w = torch.cat([x_w * torch.cos(theta_w), x_w * torch.sin(theta_w)], dim=1)
        x_d = torch.cat([x_d * torch.cos(theta_d), x_d * torch.sin(theta_d)], dim=1)
        h = self.tfc_h(x_h)
        w = self.tfc_w(x_w)
        d = self.tfc_d(x_d)
        c = self.fc_c(x)
        a = F.adaptive_avg_pool3d(h + w + d + c, output_size=1)
        a = self.reweight(a).reshape(B, self.out_dim, 4).permute(2, 0, 1).softmax(dim=0).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
        x = h * a[0] + w * a[1] + d * a[2] + c * a[3]
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class WaveBlock3D(nn.Module):
    def __init__(self, in_dim, out_dim, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.GroupNorm, mode='fc'):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        if norm_layer == nn.GroupNorm:
            self.norm1 = norm_layer(num_groups=in_dim, num_channels=in_dim)
            self.norm2 = norm_layer(num_groups=out_dim, num_channels=out_dim)
        else:
            self.norm1 = norm_layer(in_dim)
            self.norm2 = norm_layer(out_dim)
        self.attn = PATM3D(in_dim=in_dim, out_dim=out_dim, qkv_bias=qkv_bias, qk_scale=None, attn_drop=attn_drop, mode=mode)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        mlp_hidden_dim = int(out_dim * mlp_ratio)
        self.mlp = Mlp3D(in_features=out_dim, hidden_features=mlp_hidden_dim, out_features=out_dim, act_layer=act_layer)
        # Projection for residual connection if in_dim != out_dim
        self.residual_proj = nn.Conv3d(in_dim, out_dim, 1, 1) if in_dim != out_dim else nn.Identity()
    
    def forward(self, x):
        x = self.residual_proj(x) + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

In [ ]:
class Unet(nn.Module):
    def __init__(self, in_channels=4, n_channels=64, n_classes=2, drop_path_rate=0.1):
        super().__init__()
        self.in_channels = in_channels
        self.n_classes = n_classes
        self.n_channels = n_channels
        
        self.conv = DoubleConv(in_channels, 4*n_channels)
        self.enc1 = Down2W(4*n_channels, 6*n_channels)
        self.enc2 = Down2W(6*n_channels, 8*n_channels)
        self.enc3 = Down2W(8*n_channels, 12*n_channels)
        
        self.wave_enc = WaveBlock3D(
            in_dim=12*n_channels, out_dim=16*n_channels, mlp_ratio=4., qkv_bias=True,
            drop=0., attn_drop=0., drop_path=drop_path_rate / 3, norm_layer=nn.GroupNorm, mode='fc'
        )
        self.wave_bottleneck = WaveBlock3D(
            in_dim=16*n_channels, out_dim=32*n_channels, mlp_ratio=4., qkv_bias=True,
            drop=0., attn_drop=0., drop_path=drop_path_rate / 3, norm_layer=nn.GroupNorm, mode='fc'
        )
        self.bottleneck_norm = nn.GroupNorm(num_groups=32*n_channels, num_channels=32*n_channels)
        self.wave_dec = WaveBlock3D(
            in_dim=32*n_channels, out_dim=16*n_channels, mlp_ratio=4., qkv_bias=True,
            drop=0., attn_drop=0., drop_path=drop_path_rate / 3, norm_layer=nn.GroupNorm, mode='fc'
        )
        
        self.dec1 = Up1W(16*n_channels, 8*n_channels)
        self.dec2 = Up1W(8*n_channels, 6*n_channels)
        self.dec3 = Up1W(6*n_channels, 4*n_channels)
        self.out = nn.Conv3d(in_channels=4*n_channels, out_channels=n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.conv(x)
        x2, subbands1 = self.enc1(x1)
        x3, subbands2 = self.enc2(x2)
        x4, subbands3 = self.enc3(x3)
        
        x4 = self.wave_enc(x4)
        x5 = self.wave_bottleneck(x4)
        x5 = self.bottleneck_norm(x5)
        x5 = self.wave_dec(x5)
        
        x_out = self.dec1(x5, x3, subbands3)
        x_out = self.dec2(x_out, x2, subbands2)
        x_out = self.dec3(x_out, x1, subbands1)
        out = self.out(x_out)
        return out

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.GroupNorm, nn.LayerNorm)):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv3d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.kernel_size[2] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

In [ ]:
model = Unet(in_channels=4, n_classes=3, n_channels=8).to('cuda')
print('Number of network parameters:', sum(param.numel() for param in model.parameters()))

In [ ]:
print(model)

In [ ]:
# input = torch.randint(
#         low=0,
#         high=255,
#         size=(1, 4, 128, 128, 128),
#         dtype=torch.float,
# )
# input = input.to("cuda")
# output = model(input)
# print(output.shape)

## For the loss



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, gamma=0.75, smooth=1e-6):
        super(FocalTverskyLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = smooth

    def forward(self, y_pred, y_true):
        # Apply sigmoid if raw logits
        y_pred = torch.sigmoid(y_pred)

        # Flatten
        y_true = y_true.contiguous().view(-1)
        y_pred = y_pred.contiguous().view(-1)

        # Compute Tversky components
        TP = (y_true * y_pred).sum()
        FN = (y_true * (1 - y_pred)).sum()
        FP = ((1 - y_true) * y_pred).sum()

        tversky = (TP + self.smooth) / (TP + self.alpha * FN + (1 - self.alpha) * FP + self.smooth)
        focal_tversky = torch.pow((1 - tversky), self.gamma)

        return focal_tversky


class EDiceLoss(nn.Module):
    """Dice loss tailored to Brats need.
    """

    def __init__(self, do_sigmoid=True):
        super(EDiceLoss, self).__init__()
        self.do_sigmoid = do_sigmoid
        self.labels = ["ET", "TC", "WT"]
        self.device = "cpu"

    def binary_dice(self, inputs, targets, label_index, metric_mode=False):
        smooth = 1e-5
        if self.do_sigmoid:
            inputs = torch.sigmoid(inputs)

        if metric_mode:
            inputs = inputs > 0.5
            if targets.sum() == 0:
                print(f"No {self.labels[label_index]} for this patient")
                if inputs.sum() == 0:
                    return torch.tensor(1., device="cuda")
                else:
                    return torch.tensor(0., device="cuda")
            # Threshold the pred
        intersection = EDiceLoss.compute_intersection(inputs, targets)
        if metric_mode:
            dice = (2 * intersection) / ((inputs.sum() + targets.sum()) * 1.0)
        else:
            dice = (2 * intersection + smooth) / (inputs.pow(2).sum() + targets.pow(2).sum() + smooth)
        if metric_mode:
            return dice
        return 1 - dice

    @staticmethod
    def compute_intersection(inputs, targets):
        intersection = torch.sum(inputs * targets)
        return intersection

    def forward(self, inputs, target):
        dice = 0
        ce = 0
        CE_L = torch.nn.BCELoss()
        for i in range(target.size(1)):
            dice = dice + self.binary_dice(inputs[:, i, ...], target[:, i, ...], i)
            ce = ce + CE_L(torch.sigmoid(inputs[:, i, ...]), target[:, i, ...])
        final_dice = (dice + ce) / target.size(1)
        return final_dice

    def metric(self, inputs, target):
        dices = []
        for j in range(target.size(0)):
            dice = []
            for i in range(target.size(1)):
                dice.append(self.binary_dice(inputs[j, i], target[j, i], i, True))
            dices.append(dice)
        return dices


class EDiceLoss_Val(nn.Module):
    """Dice loss tailored to Brats need.
    """

    def __init__(self, do_sigmoid=True):
        super(EDiceLoss_Val, self).__init__()
        self.do_sigmoid = do_sigmoid
        self.labels = ["ET", "TC", "WT"]
        self.device = "cpu"

    def binary_dice(self, inputs, targets, label_index, metric_mode=False):
        smooth = 1e-5
        if self.do_sigmoid:
            inputs = torch.sigmoid(inputs)

        if metric_mode:
            inputs = inputs > 0.5
            if targets.sum() == 0:
                print(f"No {self.labels[label_index]} for this patient")
                if inputs.sum() == 0:
                    return torch.tensor(1., device="cuda")
                else:
                    return torch.tensor(0., device="cuda")
            # Threshold the pred
        intersection = EDiceLoss_Val.compute_intersection(inputs, targets)
        if metric_mode:
            dice = (2 * intersection) / ((inputs.sum() + targets.sum()) * 1.0)
        else:
            dice = (2 * intersection + smooth) / (inputs.pow(2).sum() + targets.pow(2).sum() + smooth)
        if metric_mode:
            return dice
        return 1 - dice

    @staticmethod
    def compute_intersection(inputs, targets):
        intersection = torch.sum(inputs * targets)
        return intersection

    def forward(self, inputs, target):
        dice = 0
        for i in range(target.size(1)):
            dice = dice + self.binary_dice(inputs[:, i, ...], target[:, i, ...], i)
        final_dice = dice / target.size(1)
        return final_dice

    def metric(self, inputs, target):
        dices = []
        for j in range(target.size(0)):
            dice = []
            for i in range(target.size(1)):
                dice.append(self.binary_dice(inputs[j, i], target[j, i], i, True))
            dices.append(dice)
        return dices

class CombinedLoss(nn.Module):
    def __init__(self, use_focal_tversky=True):
        super(CombinedLoss, self).__init__()
        self.focal_tversky = FocalTverskyLoss()
        self.dice = EDiceLoss()
        self.use_focal_tversky = use_focal_tversky

    def forward(self, inputs, targets):
        loss = self.dice(inputs, targets)
        if self.use_focal_tversky:
            loss += self.focal_tversky(inputs, targets)
        return loss


class AverageMeter(object):
    """Computes and stores the average and current value."""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    @staticmethod
    def _get_batch_fmtstr(num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

# ===================================================================================================

### Traing CoT+DA

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
WANDB_KEY = user_secrets.get_secret("WANDB_KEY")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
post_trans = Compose(
    [EnsureType(), Activations(sigmoid=True), AsDiscrete(argmax=False, threshold=0.5)]
)

post_sigmoid = Activations(sigmoid=True)
post_pred = AsDiscrete(argmax=False, threshold=0.5)
roi = (128, 128, 128) #128, 128, 128
sw_batch_size = 1
overlap = 0.5
VAL_AMP = True

os.environ["WANDB_DIR"] = os.path.abspath("/kaggle/working")

wandb.login(key = WANDB_KEY)

In [ ]:

#----------------
def train_epoch(model, loader, optimizer, epoch, loss_func, end, wandb_tracking = 0):
    model.train()
    run_loss = AverageMeter('Loss', ':.4e')

    num_steps = len(loader)
    print(f"Epoch {epoch}: Number of steps (batches) in this epoch: {num_steps}")

    for idx, batch_data in enumerate(loader):
        torch.cuda.empty_cache()
        data, target = batch_data["image"].float().cuda(), batch_data["label"].float().cuda()
        logits = model(data)

        loss = loss_func(logits, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        run_loss.update(loss.item(), n=batch_size)
        if wandb_tracking == 1:
            wandb.log({"train_step_loss": loss.item()})

    # Log the average loss for the epoch
    if wandb_tracking == 1:
        wandb.log({"train_epoch_loss": run_loss.avg, "epoch": epoch})
    return run_loss.avg

# ===============================================================================================
def model_inferer(input, model):
    def _compute(input):
        return sliding_window_inference(
            inputs=input,
            roi_size= roi,
            sw_batch_size=sw_batch_size,
            predictor=model,
            overlap=overlap,
        )
    with torch.no_grad(), torch.amp.autocast(device_type='cuda', enabled=VAL_AMP):
        output = _compute(input)
    
    # Giải phóng bộ nhớ không cần thiết
    torch.cuda.empty_cache()
    return output


# ===============================================================================================

def evaluate_model(model, loader, epoch, acc_func, criterian_val, metric, wandb_tracking = 0):
    model.eval()
    run_acc = AverageMeter('Loss', ':.4e')
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for idx, batch_data in enumerate(loader):
            val_inputs, val_labels = batch_data["image"].to(device), batch_data["label"].to(device)

            logits = model_inferer(val_inputs, model)
            val_outputs_list = decollate_batch(logits)
            val_labels_list = decollate_batch(val_labels)


            # Ensure all tensors are on the same device
            val_output_convert = [post_trans(val_pred_tensor).to(device) for val_pred_tensor in val_outputs_list]
            val_labels_convert = [label.to(device) for label in val_labels_list]
            del val_inputs, val_labels, logits, val_outputs_list, val_labels_list
            torch.cuda.empty_cache()
            all_preds.extend(val_output_convert)
            all_labels.extend(val_labels_convert)

            # Reset and compute metrics
            acc_func.reset()
            acc_func(y_pred=val_output_convert, y=val_labels_convert)

            acc, not_nans = acc_func.aggregate()
            run_acc.update(acc.cpu().numpy(), n=not_nans.cpu().numpy())
        
        dice_et = run_acc.avg[0]
        dice_tc = run_acc.avg[1]
        dice_wt = run_acc.avg[2]

    # Log validation metrics
    if wandb_tracking == 1:
        wandb.log({
            "val_epoch_dice_et": dice_et,
            "val_epoch_dice_tc": dice_tc,
            "val_epoch_dice_wt": dice_wt,
            "epoch": epoch
        })
    return run_acc.avg

# ===============================================================================================


def trainer(model, train_loader, val_loader, optimizer, loss_func, 
            acc_func, criterian_val, metric, scheduler, start_epoch=1, end_epoch=3, save_every=1, checkpoint_path=None, wandb_tracking = 0):
    # Initialize wandb logging
    if wandb_tracking:
        wandb.init(entity="huynhtongdangkhoa-vietnam-national-university", project="AblationStudy", name="Baseline_E_seed1", id="19633214152", resume="allow", config={
            "epochs": end_epoch,
            "optimizer": optimizer.__class__.__name__,
            "learning_rate": optimizer.param_groups[0]["lr"],
            "loss_func": loss_func.__class__.__name__,
            "scheduler": scheduler.__class__.__name__ if scheduler else None
        })
        wandb_table = wandb.Table(columns=["Epoch", "Train Loss", "Dice_ET", "Dice_TC", "Dice_WT", "Best Model"])
        print("Wandb ID:", wandb.run.id)
        wandb_table = wandb.Table(columns=["Epoch", "Train Loss", "Dice_ET", "Dice_TC", "Dice_WT", "Best Model"])
        print("Wandb ID:", wandb.run.id)

    val_acc_max = 0.0
    best_epoch = 0
    TC_dices = []
    WT_dices = []
    ET_dices = []
    avg_dices = []
    loss_epochs, train_epochs = [], []

    # If checkpoint path is provided, load model and optimizer states
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, weights_only = False)
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        if scheduler:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # Start from the next epoch
        val_acc_max = checkpoint['val_acc_max']  # Retain the best validation accuracy

        print(f"Resuming training from epoch {start_epoch}...")
    
    for epoch in range(start_epoch, end_epoch + 1):
        # Train the model for the current epoch
        train_loss = train_epoch(model, train_loader, optimizer, epoch=epoch, loss_func=loss_func, end=end_epoch, wandb_tracking = wandb_tracking)
        torch.cuda.empty_cache()

        # Update scheduler if available
        if scheduler is not None:
            scheduler.step()
        
        # Evaluate the model after every 'save_every' epoch or at the last epoch
        if epoch % save_every == 0 or epoch == end_epoch or epoch == start_epoch:
            loss_epochs.append(train_loss)
            train_epochs.append(epoch)

            val_acc = evaluate_model(model, val_loader, epoch=epoch,
                                     acc_func=acc_func, criterian_val=criterian_val, metric=metric, wandb_tracking = wandb_tracking)
            ET_dice = val_acc[0]
            TC_dice = val_acc[1]
            WT_dice = val_acc[2]
            val_avg_acc = np.mean(val_acc)

            # Update dice coefficients
            ET_dices.append(ET_dice)
            TC_dices.append(TC_dice)
            WT_dices.append(WT_dice)
            avg_dices.append(val_avg_acc)

            # Check if this is the best model
            best_model_flag = False
            if val_avg_acc > val_acc_max:
                print(f"New best ({val_acc_max:.6f} --> {val_avg_acc:.6f}) at epoch {epoch}")
                val_acc_max = val_avg_acc
                best_epoch = epoch
                best_model_flag = True
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),  # Save scheduler state
                'val_acc_max': val_acc_max,
            }, save_current)  # Save model to 'model.pth'

            if best_model_flag:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),  # Save scheduler state
                    'val_acc_max': val_acc_max,
                }, save_path)  # Save model to 'model.pth'
                

            torch.cuda.empty_cache()

            # ✅ Add row to wandb table
            if wandb_tracking == 1:
                wandb_table.add_data(epoch, train_loss, ET_dice, TC_dice, WT_dice, "Yes" if best_model_flag else "No")

    # ✅ Log table to wandb
    if wandb_tracking == 1:
        wandb.log({"Training Metrics": wandb_table})

    return (val_acc_max, TC_dices, WT_dices, ET_dices, avg_dices, loss_epochs, train_epochs)

In [ ]:
start = 0
end = 200 #max_epochs
batch_size = 1
save_every = 5 #

learning_rate = 1e-4
weight_decay = 1e-5
checkpoint_path = "/kaggle/input/doublew-hf/pytorch/default/1/DoubleW-hf.pth"
###
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "best_metric_model.pth")
save_current = os.path.join(save_dir, "current_model.pth")
###

criterion = CombinedLoss().cuda()
criterian_val = EDiceLoss_Val().cuda()
metric = criterian_val.metric

dice_acc = DiceMetric(include_background=True, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = end)


(val_acc_max, TC_dices, WT_dices, ET_dices, avg_dices, loss_epochs, train_epochs)  = trainer(model = model,
                                                                                            train_loader = train_loader,
                                                                                            val_loader = val_loader,
                                                                                            optimizer = optimizer,
                                                                                            loss_func = criterion,
                                                                                            acc_func = dice_acc,
                                                                                            criterian_val = criterian_val,
                                                                                            metric = metric,
                                                                                            scheduler = scheduler,
                                                                                            start_epoch = start,
                                                                                            end_epoch = end,
                                                                                            save_every = save_every,
                                                                                            checkpoint_path = checkpoint_path,
                                                                                            wandb_tracking = 1)